# <font color='red'>Geoplatform - Project Setup</font>
<b><u>Contents:</b></u><br>
1) Register a service URL to the specified Portal<br>
2) Create Geoplatform group<br>
3) Add service item to group(s)<br>
4) Note on making field map copy<br>
<font color='blue' size='2'>Note:  Always run cells with blue headers before running any other processes. Many cells depend on variables or output from previous cells.</font>

# <font color=blue>Import modules</font>

In [ ]:
import os, sys
import arcgis
from arcgis.gis import GIS
from arcgis.mapping import WebMap
import getpass
from colorama import Fore, Back, Style
print(Fore.BLUE+"Modules Imported "+Style.RESET_ALL+str(datetime.now().strftime("%m/%d/%Y %H:%M "))+(time.localtime().tm_zone))

# <font color=blue>Project variables</font>

In [ ]:
### INPUT CELL ###

# Target portal URL
portalURL = 'https://epa.maps.arcgis.com/'

# Specify a project name:
projectName = 'Project Name'

# Specify tag(s) for items:
projectTags = ['Project Tag,EPA,R9,START']

### INPUT SECTION END ###

# <font color=blue>Connect to Portal</font>
- <font color=red>Important:</font>  Connection method in this script requires you to have ArcGIS Pro logged into the correct ArcGIS Online Organization target (U.S. EPA Geoplatform).

In [ ]:
### ArcGIS portal url and login ###
portal = GIS('pro')
token = portal._con.token
print(Fore.BLUE+"Logged in as: "+Style.RESET_ALL+str(portal.properties.user.username))

# 1) Register a service URL to the specified Portal
- Creates an item in Geoplatform referencing an ERTFOS rest URL
- Adds project tags and a description

In [ ]:
### INPUT PARAMETERS CELL ###

# Name for the item in Geoplatform:
serviceTitle = 'R9 Project Mobile Service' # 'enter a name for the service item to be registered in Geoplatform'

# URL of the service in ERTFOS:
serviceURL = 'https://r9data....../FeatureServer' # 'paste rest URL for feature service here'

# Item description:
serviceDescription = 'Service description text' # 'enter item description here'

# Item snippet:
serviceSnippet = "Item snippet text"

### INPUT SECTION END ###

In [ ]:
register_properties = {
"title":serviceTitle,
"tags":projectTags,
"url":serviceURL,
"type":"Feature Service",
"description":serviceDescription,
"snippet":serviceSnippet,
"serviceUsername":"ServerManagerUsername",
"servicePassword":getpass.getpass(Fore.RED+"Enter server manager password to be stored in item:"+Style.RESET_ALL),
"token":token
}
newLyr_Item = portal.content.add(data=ServiceURL,item_properties=register_properties, folder=output_folder,
                                            thumbnail = GroupImage)
newLyr_Item
newLyr_ItemID = newLyr_Item.id
print(Fore.BLUE+"Item ID:  "+Style.RESET_ALL+newLyr_ItemID)

# 2) Create Geoplatform group
- Creates new Geoplatform group(s) based on a list of suffixes you provide
- Adds project tags and description to the group
- Adds default list of usernames to the group
- Turns on deletion projection for the new group(s)
- Reassigns ownership of the group(s) to the GIS user specified in the "Move item" section

In [ ]:
### INPUT PARAMETER CELL ###

# Can specify a list of groups, each list name in variable groupList will create a group with project name and suffix:
groupList = ['Group'] # Default is 'Group' for just one project group, or list other suffixes for other groups.

# Provide a description:
groupDescription = 'Description text for group created using the API for Python'

# Group snippet:
groupSnippet = 'Snippet text for group'

# Base user list:
baseUserList = ['User.Name1_EPAEXT','User.Name2_EPAEXT','User.Name3_EPAEXT', \
                'User.Name4_EPAEXT','User.Name5_EPAEXT','User.Name6_EPAEXT']

# Filepath to an image file for the group:
groupImage = r'E:/ContractFolder/ProjectFolder/Data/Project_Data/Rasters/GeoPlatformGroupPic.JPG' # example: r'C:/PythonTemp/TestImage.jpg'

### INPUT SECTION END ###

In [ ]:
def create_groups():
shareGroupList = [] # Blank list for appending group id's    
    for groupType in groupList:
        geocaching_group = portal.groups.create(title= projectName+' '+groupType,
                                            tags = projectTags,
                                            description = projectName+': '+groupDescription,
                                            snippet = projectName+': '+groupSnippet,
                                            access = 'private',
                                            is_invitation_only = 'True',
                                            # protected=True,
                                            thumbnail = groupImage)
        geocaching_group.add_users(baseUserList)
        # geocaching_group.reassign_to(moveToUserName)
        print('CREATED:  '+Fore.BLUE+projectName+' '+groupType+Style.RESET_ALL+'  GROUP ID:  '+Fore.BLUE+geocaching_group.groupid+Style.RESET_ALL)
        shareGroupList.append(geocaching_group.groupid)
create_groups()
print(Fore.GREEN+"END"+Style.RESET_ALL)

# 3) Share new service item to group(s)

In [ ]:
### INPUT PARAMETERS CELL ###

groupMembersEditItem = False # Sets whether group members can edit the item.

### INPUT SECTION END ###

In [ ]:
newLyr_Item.share(everyone=False,org=False,groups=shareGroupList,allow_members_to_edit=groupMembersEditItem)

# 4) Optional - Next step use the Copy Web Map Script
- if saving time by starting with another web map as a template for symbology, smart forms, etc.